# Environmental Mapping Playground

In [1]:
import geemap

In [20]:
nh_center = [43.78699687528447, -71.51658744025995]
nh_zoom = 8

In [21]:
m = geemap.Map(center=nh_center, zoom=nh_zoom)
m.add_basemap("HYBRID")
hydro_url = "https://basemap.nationalmap.gov/arcgis/services/USGSHydroCached/MapServer/WMSServer?"
m.add_wms_layer(url=hydro_url,
                layers="0",  # Find all options in XML
                name="USGS Hydro",
                format="image/png",
                shown=True)
m

Map(center=[43.78699687528447, -71.51658744025995], controls=(WidgetControl(options=['position', 'transparent_…

In [24]:
m2 = geemap.Map(center=nh_center, zoom=nh_zoom)
# m2.basemap_demo()
m2.add_basemap("HYBRID")
m2.add_basemap("NLCD 2016 CONUS Land Cover")
m2

Map(center=[43.78699687528447, -71.51658744025995], controls=(WidgetControl(options=['position', 'transparent_…